In [ ]:
import os, shutil

In [ ]:
data_base_path = "/home/erbl/data/roboflow"
os.chdir(data_base_path)

In [ ]:
!pwd

In [ ]:
# roboflow 데이터 다운받기
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="X2gvBX7Me32fy3mU0Fkz")
project = rf.workspace("ebrlhfr-interaction").project("interaction-nnzcz")
dataset = project.version(1).download("darknet")

# 새로 만든 데이터 N배 만들기

In [ ]:
interaction_dataset_base_path = f"{data_base_path}/Interaction-1" # 수정 필요

In [ ]:
train_dataset_base_path = f"{interaction_dataset_base_path}/train"
valid_dataset_base_path = f"{interaction_dataset_base_path}/valid"

In [ ]:
def get_filename_list(base_path, *extension):
	all_filenames = os.listdir(base_path)
	
	if len(extension) > 0:
		result = []
		for ex in extension:
			result += [".".join(filename.split(".")[:-1]) for filename in all_filenames if filename.endswith(f".{ex}")]
		return result

	return all_filenames


In [ ]:
train_image_filename_list = get_filename_list(train_dataset_base_path, "jpg")
valid_image_filename_list = get_filename_list(valid_dataset_base_path, "jpg")


In [ ]:
# 확인
print(len(train_image_filename_list))
print(len(valid_image_filename_list))

print(train_image_filename_list[:10])
print(valid_image_filename_list[:10])


In [ ]:
# 데이터셋 N배 증가하는 함수

def copy_files_N_times(source_path, destination_path, *N):
    filename_list = get_filename_list(source_path, "jpg")
    if len(N): N = N[0]
    else: N = 1
    for index in range(N):
        for filename in filename_list:
            shutil.copyfile(f"{source_path}/{filename}.jpg", f"{destination_path}/{filename}_{index}.jpg")
            shutil.copyfile(f"{source_path}/{filename}.txt", f"{destination_path}/{filename}_{index}.txt")

In [ ]:
N = 8
new_dataset_base_path = f"{interaction_dataset_base_path}_{N}_times"

In [ ]:

os.makedirs(new_dataset_base_path, exist_ok=True)
os.makedirs(f"{new_dataset_base_path}/train", exist_ok=True)
os.makedirs(f"{new_dataset_base_path}/valid", exist_ok=True)

In [ ]:
copy_files_N_times(train_dataset_base_path, f"{new_dataset_base_path}/train", N)
copy_files_N_times(valid_dataset_base_path, f"{new_dataset_base_path}/valid", N)

In [ ]:
print(1974 * N)
print(168 * N)

# Person dataset annotation 값 +2, copy to interation-1_8_times

In [ ]:
# 기존의 person dataset copy
person_dataset_path = f"{data_base_path}/EBRL_Person-1"
destination_dataset_path = new_dataset_base_path

In [ ]:
f"{person_dataset_path}/valid"

In [ ]:
def copy_and_increase_annotation_index(source_path, destination_path, number):
    filename_list = get_filename_list(source_path, "txt")
    print(len(filename_list))
    for filename in filename_list:
        # JPG 그대로 복사
        shutil.copyfile(f"{source_path}/{filename}.jpg", f"{destination_path}/{filename}.jpg")

        # TXT ANNOTATION number 만큼 증가
        with open(f"{source_path}/{filename}.txt", "r") as f:
            lines = f.readlines()
        with open(f"{destination_path}/{filename}.txt", "w") as f:
            for line in lines:
                line = str(int(line[0]) + number) + line[1:]
                f.write(line)

In [ ]:
copy_and_increase_annotation_index(f"{person_dataset_path}/valid", f"{destination_dataset_path}/valid", 2)

In [ ]:
copy_and_increase_annotation_index(f"{person_dataset_path}/train", f"{destination_dataset_path}/train", 2)

(Interaction-1_8_times) train: 15792, valid: 1344  
(EBRL_Person-1) train: 15466, valid: 1326

(Interaction-1_8_times) train: 31258, valid: 2670
